In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [0]:

import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [0]:
train = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv')

In [0]:
train.shape

In [0]:
train.isnull().sum()

In [0]:
#Data cleaning

import re

def text_process(text):
    '''Make text lowercase, remove text in square brackets,remove all single character, Substituting multiple spaces with single space,removing all special characters,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]#', '', text)
    #text = re.sub(r'\W', ' ', str(X[text]))
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    #text = re.sub('[%s]' % re.escape(str.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('©', '', text)
    text = re.sub('@', '', text)
    text = re.sub('#', '', text)
    text = re.sub('ûò', '', text)
    text = re.sub('!', '', text)
    text = re.sub('&', '', text)
 
    #text = re.sub('?', '', text)
    return text

In [0]:
train['comment_text'] = train['comment_text'].apply(lambda x: text_process(x))

In [0]:
train = train.rename(columns={"comment_text": "text"})

In [0]:
#Validation dataset

valid = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv')

In [0]:
valid.head()

In [0]:
# Applying the cleaning function 
valid['comment_text'] = valid['comment_text'].apply(lambda x: text_process(x))

In [0]:
valid = valid.rename(columns={"comment_text": "text"})

In [0]:
#Load test dataset

test = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv')

In [0]:
test.head()

In [0]:
# Applying the cleaning function 
test['comment_text'] = test['comment_text'].apply(lambda x: text_process(x))

In [0]:
test = test.rename(columns={"comment_text": "text"})

In [0]:
!pip install transformers

In [0]:
#Bert Model

import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [0]:
#Function for encoding the word/comment to integer or vector

def regular_encode(texts, tokenizer, maxlen=512):
  enc_di = tokenizer.batch_encode_plus(
      texts,
      return_attention_masks=False,
      return_token_type_ids=False,
      pad_to_max_length=True,
      max_length=maxlen
  )

  return np.array(enc_di['input_ids'])

In [0]:
#BUILD MODEL FUNCTION

def build_model(transformer, max_len=512):
  input_word_ids = Input(shape=max_len, dtype=tf.int32, name='input_word_ids')
  sequence_output=transformer(input_word_ids)[0]
  cls_token=sequence_output[:, 0, :]
  out = Dense(1, activation='sigmoid')(cls_token)

  model = Model(inputs=input_word_ids, outputs=out)
  model.compile(Adam(lr=1e-5), loss='binary_crossentropy',metrics=[tf.keras.metrics.AUC()])

  return model

In [0]:
EPOCHS = 10
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync 
MAX_LEN = 512  
MODEL = 'bert-base-multilingual-cased'

In [0]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [0]:
#Encode comments (text into vectors)

x_train = regular_encode(train.text.values, tokenizer, maxlen=MAX_LEN)

x_valid = regular_encode(valid.text.values, tokenizer, maxlen=MAX_LEN)

y_train = train.toxic.values

y_valid = valid.toxic.values

In [0]:
x_test = regular_encode(test.text.values, tokenizer, maxlen=MAX_LEN)

In [0]:
AUTO = tf.data.experimental.AUTOTUNE

In [0]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)    
    .batch(BATCH_SIZE)
    .prefetch(AUTO) 
)

In [0]:
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))  #no shuffle or repeat
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

In [0]:
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test) 
    .batch(BATCH_SIZE)
)

In [0]:
#Model

In [0]:
MODEL = 'bert-base-multilingual-cased'

In [0]:
#Build model from BERT pretrained model and the build model function

with tpu_strategy.scope():   #enables to use the TPU while training the model
  transformer_layer = TFAutoModel.from_pretrained(MODEL)
  model = build_model(transformer_layer, max_len = MAX_LEN)

In [0]:
model.summary()

In [0]:
!pip install h5py

In [0]:
#Call backs

EPOCHS = 2

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler

#stop = EarlyStopping(monitor='val_auc', mode='max', min_delta=0.01, patience=1, verbose=1)

#rlrop = ReduceLROnPlateau(monitor='val_auc', mode='auto', min_delta=0.01, factor=0.2, patience=1) 

#filepath = '/kaggle/working/est_bert_model_tpu_v2.hdf5'   # Saved model checkpoint file path

#checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max', save_freq='epoch')

In [0]:
start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005 * tpu_strategy.num_replicas_in_sync

rampup_epochs = 5
sustain_epochs = 0
exp_decay=0.8

def lrfn(epoch):
  if epoch < rampup_epochs:
    return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
  elif epoch < rampup_epochs + sustain_epochs:
    return max_lr
  else:
    return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr

lr_callback = LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True)

rang = np.arange(EPOCHS)
y = [lrfn(x) for x in rang]
plt.plot(rang, y)

print('Learning rate per epoch:')

In [0]:
#Model training

n_steps = x_train.shape[0] // BATCH_SIZE

model.fit(train_dataset, steps_per_epoch=n_steps, validation_data=valid_dataset, epochs=2, callbacks=[lr_callback])

In [0]:
n_steps = x_valid.shape[0] // BATCH_SIZE
model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=2
)

In [0]:
sub = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [0]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission_bert_tpu.csv', index=False)